# Building Q-Table for Choosing Cards

In [1]:
from card_choice_gym import CardChoiceEnv
from utils import save_q_table, write_calls
import numpy as np
from keras.models import load_model
import time
import os
from collections import Counter
from tqdm import tqdm

/usr/local/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [10]:
q = np.load('../models/q-table.npy')

In [2]:
if os.path.exists('../models/calling_nn_1.pickle'):
    cm = load_model('../models/calling_nn_1.pickle')


2022-05-16 21:50:22.246160: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def LearnJoker(q_in=np.zeros((4, 10, 3, 4, 19, 36)), alpha_in=0.01, epsilon_in=0.5, gamma_in=0.95, episodes_in=100, cm_in=None):
  env = CardChoiceEnv()

  if cm_in == None:
    print("NO CM")
  else:
    env.calling_model = cm_in

  alpha, gamma, epsilon = alpha_in, gamma_in, epsilon_in
  q = q_in
  
  wins = []
  good_calls = np.zeros((7,))

  actions = []

  for i in tqdm(range(episodes_in)): 
    done = False
    s = env.reset()
    action_value_list= q[s]

    while True: 
      if np.random.random() < epsilon:
        action = env.action_space.sample()
        if not env.in_playable(0, action):
          action = env.action_space.sample()
      else:
        for poss, _ in Counter({a : b for (a, b) in enumerate(action_value_list)}).most_common():
          if env.in_playable(0, poss):
            action = poss 
    
      s_, r, done, _ = env.step(action)
      td_target = r + gamma * np.max(q[s_])
      td_error = td_target - q[s][action]
      q[s][action] += alpha * td_error
      s = s_
      if done:
        if r > 0:
          wins.append(i)
          good_calls = np.vstack((good_calls, [env.call_state]))
        break
      actions.append(action)
  return wins, good_calls, q, actions

In [19]:
%reload_ext autoreload
%autoreload 2

In [10]:
eps = 50000
wins, calls, q, acts= LearnJoker(epsilon_in=1, episodes_in=eps, alpha_in=0.01, gamma_in=1, cm_in=cm)
print(f"Win rate: {len(wins)/ eps}")


100%|██████████| 50000/50000 [01:49<00:00, 456.21it/s]

Win rate: 0.34244


In [12]:
eps = 1000
wins, calls, q, acts= LearnJoker(epsilon_in=0, episodes_in=eps, alpha_in=0.01, gamma_in=1, q_in=q, cm_in=cm)
print(f"Win rate: {len(wins)/ eps}")

100%|██████████| 1000/1000 [00:03<00:00, 278.35it/s]

Win rate: 0.216


In [8]:
eps = 80000
wins, calls, q, acts= LearnJoker(epsilon_in=0.5, episodes_in=eps, alpha_in=0.0001, gamma_in=1)
print(f"Win rate: {len(wins)/ eps}")

100%|██████████| 80000/80000 [04:01<00:00, 331.82it/s]

Win rate: 0.3057875


In [18]:
eps = 1000
ep = 1
while ep >= 0:
    wins, calls, q, acts= LearnJoker(epsilon_in=ep, episodes_in=eps, alpha_in=0.0001, gamma_in=1)
    print(f"Win rate: {len(wins)/ eps}")
    ep -= 0.05

100%|██████████| 1000/1000 [00:02<00:00, 472.38it/s]


Win rate: 0.347


100%|██████████| 1000/1000 [00:02<00:00, 438.02it/s]


Win rate: 0.344


100%|██████████| 1000/1000 [00:02<00:00, 451.00it/s]


...

Win rate: 0.27


100%|██████████| 1000/1000 [00:03<00:00, 319.60it/s]


Win rate: 0.247


100%|██████████| 1000/1000 [00:03<00:00, 315.01it/s]

Win rate: 0.269


In [4]:
import cProfile
import pstats

eps = 10000
with cProfile.Profile() as pr:
    LearnJoker(epsilon_in=0.5, episodes_in=eps, alpha_in=0.0001, gamma_in=1, cm_in=cm)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
# stats.print_stats()
stats.dump_stats(filename='10k1.prof')


  0%|          | 0/10000 [00:00<?, ?it/s]


IndexError: pop from empty list

In [ ]:
from more_itertools import quantify
for i in range(36):
    print(acts.count(i))

In [ ]:
save_q_table(q)
write_calls(calls)

### Speeding up playable()

In [27]:
from joker import Card
playable = [Card(4, 0), Card(4, 1), Card(4, 1), Card(4, 2), Card(4, 3)]

In [29]:
other = Card(4,0)
other in playable

True

In [38]:

import random
# a = time.perf_counter()
all = []
for i in range(9):
    all.append(Card(random.randint(4,13),random.randint(0,3)))
a = time.perf_counter()
for i in range(1000000):
    random.shuffle(all)
    suit = random.randint(0,3)
    firsts = list(filter(lambda x: x.suit == suit, all))
b = time.perf_counter()
b - a


5.601851543000521

[(2, 3), (2, 3), (2, 3), (2, 3), (2, 3), (2, 3), (2, 3), (2, 3), (2, 3)]

In [40]:
[(Card(i, j)) for j in range(0,4) for i in range(4, 14)]

[Six of Diamonds,
 Seven of Diamonds,
 Eight of Diamonds,
 Nine of Diamonds,
 Ten of Diamonds,
 Jack of Diamonds,
 Queen of Diamonds,
 King of Diamonds,
 Ace of Diamonds,
 Joker,
 Six of Clubs,
 Seven of Clubs,
 Eight of Clubs,
 Nine of Clubs,
 Ten of Clubs,
 Jack of Clubs,
 Queen of Clubs,
 King of Clubs,
 Ace of Clubs,
 Joker,
 Six of Hearts,
 Seven of Hearts,
 Eight of Hearts,
 Nine of Hearts,
 Ten of Hearts,
 Jack of Hearts,
 Queen of Hearts,
 King of Hearts,
 Ace of Hearts,
 Joker,
 Six of Spades,
 Seven of Spades,
 Eight of Spades,
 Nine of Spades,
 Ten of Spades,
 Jack of Spades,
 Queen of Spades,
 King of Spades,
 Ace of Spades,
 Joker]

In [9]:
temp = np.zeros((4, 9))
rn = np.asarray(list(range(36)))
for i in range(4):
    temp[i] = np.asarray(rn.take(range((i * 9), ((i + 1) * 9))))
temp

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [ 9., 10., 11., 12., 13., 14., 15., 16., 17.],
       [18., 19., 20., 21., 22., 23., 24., 25., 26.],
       [27., 28., 29., 30., 31., 32., 33., 34., 35.]])

In [15]:
from joker import Card 
all = np.array([Card(value, suit) for value in range(4, 14) for suit in range(0, 4) if suit % 2 != 0 or (value != 4 and value != 13) ])
np.random.shuffle(all)
all.reshape(4,9)
all

ValueError: cannot reshape array of size 38 into shape (4,9)